## AI Tool Calling

Readings:

https://python.langchain.com/docs/concepts/tool_calling/

In [1]:
# Install package if not Install
# !pip install -qU langchain-google-genai

Key concepts

- Tool Creation: Use the @tool decorator to create a tool. A tool is an association between a function and its schema.
- Tool Binding: The tool needs to be connected to a model that supports tool calling. This gives the model awareness of the tool and the associated input schema required by the tool.
- Tool Calling: When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's input schema.
- Tool Execution: The tool can be executed using the arguments provided by the model.

In [2]:
# Set API Key in Windows Environment Variables 'GOOGLE_API_KEY'

import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

#### Tool Creation

In [3]:
from langchain_core.tools import tool

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

#### Tool Binding

In [4]:

llm_with_tools = model.bind_tools([multiply])

#### Tool Calling

In [5]:
result = llm_with_tools.invoke("Hello world!")

In [6]:
result

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--19c89689-e9c4-4223-980b-5deeab266565-0', usage_metadata={'input_tokens': 66, 'output_tokens': 9, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [7]:
result.tool_calls

[]

In [8]:
messages = [HumanMessage(content="What is 2 multiplied by 3?")]
ai_msg: AIMessage = llm_with_tools.invoke(messages)


In [9]:
ai_msg

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 3.0, "a": 2.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1fd234e1-35ad-477e-adc8-f84bb30e1d53-0', tool_calls=[{'name': 'multiply', 'args': {'b': 3.0, 'a': 2.0}, 'id': 'd7e1b78d-8fd1-4261-8155-9302c6d7021d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 67, 'total_tokens': 139, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 49}})

In [10]:
ai_msg.tool_calls

[{'name': 'multiply',
  'args': {'b': 3.0, 'a': 2.0},
  'id': 'd7e1b78d-8fd1-4261-8155-9302c6d7021d',
  'type': 'tool_call'}]

#### Tool Execution

In [11]:
tool_msgs = []
for call in ai_msg.tool_calls:
    if call["name"] == "multiply":
        out = multiply.invoke(call["args"])
        # Package tool output as a ToolMessage with the SAME tool_call_id
        tool_msgs.append(ToolMessage(content=str(out), tool_call_id=call["id"]))

tool_msgs

[ToolMessage(content='6', tool_call_id='d7e1b78d-8fd1-4261-8155-9302c6d7021d')]

In [12]:
# Feed the AIMessage + ToolMessage(s) back to the model to get the final answer
messages += [ai_msg] + tool_msgs

final = llm_with_tools.invoke(messages)

print(final.content)

The answer is 6.
